In [1]:
import pandas as pd
import numpy as np

In [11]:
settings_path = '~/Desktop/vmi-load-export.csv'
inventory_path = '~/Desktop/Vmi Analytics Export.csv'
save_path = '~/Desktop/refill_sheet.csv'

In [12]:
settings = pd.read_csv(settings_path)

In [13]:
inventory = pd.read_csv(inventory_path)

In [14]:
settings.head()

,Part Number,EAU,Inventory,Bag Qty,Min,Max,Lead Time
0,48-321,50000,0,100,10000,20000,16
1,48-323,75000,0,100,30000,45000,16
2,48-324,65000,0,100,10000,15000,16
3,48-325,15000,0,100,5000,7700,16
4,48-326,15000,0,100,4000,6700,16


In [15]:
settings.shape

(15, 7)

In [16]:
inventory.head()

,Part Number,EAU,Inventory,Sell,Usage,Run Rate,Weeks Left,Safety Margin,Lead Time,Use EAU
0,48-321,50000,15500,0.0917,19483,81%,19,1.0,16,NaN
1,48-323,75000,33122,0.0983,57600,160%,14,0.0,16,NaN
2,48-324,65000,14500,0.1050,7500,24%,48,3.0,16,NaN
3,48-325,15000,8700,0.1117,9300,129%,23,1.0,16,NaN
4,48-326,15000,0,0.1333,20100,221%,0,0.0,16,NaN


In [17]:
inventory.shape

(15, 10)

In [18]:
data = inventory.merge(settings, how='left', on='Part Number' )
data.head()

,Part Number,EAU_x,Inventory_x,Sell,Usage,Run Rate,Weeks Left,Safety Margin,Lead Time_x,Use EAU,EAU_y,Inventory_y,Bag Qty,Min,Max,Lead Time_y
0,48-321,50000,15500,0.0917,19483,81%,19,1.0,16,NaN,50000,0,100,10000,20000,16
1,48-323,75000,33122,0.0983,57600,160%,14,0.0,16,NaN,75000,0,100,30000,45000,16
2,48-324,65000,14500,0.1050,7500,24%,48,3.0,16,NaN,65000,0,100,10000,15000,16
3,48-325,15000,8700,0.1117,9300,129%,23,1.0,16,NaN,15000,0,100,5000,7700,16
4,48-326,15000,0,0.1333,20100,221%,0,0.0,16,NaN,15000,0,100,4000,6700,16


In [19]:
data['Exact Stocking Quantity'] = data['Max'] - data['Inventory_x']

In [20]:
data.head()

,Part Number,EAU_x,Inventory_x,Sell,Usage,Run Rate,Weeks Left,Safety Margin,Lead Time_x,Use EAU,EAU_y,Inventory_y,Bag Qty,Min,Max,Lead Time_y,Exact Stocking Quantity
0,48-321,50000,15500,0.0917,19483,81%,19,1.0,16,NaN,50000,0,100,10000,20000,16,4500
1,48-323,75000,33122,0.0983,57600,160%,14,0.0,16,NaN,75000,0,100,30000,45000,16,11878
2,48-324,65000,14500,0.1050,7500,24%,48,3.0,16,NaN,65000,0,100,10000,15000,16,500
3,48-325,15000,8700,0.1117,9300,129%,23,1.0,16,NaN,15000,0,100,5000,7700,16,-1000
4,48-326,15000,0,0.1333,20100,221%,0,0.0,16,NaN,15000,0,100,4000,6700,16,6700


In [21]:
data['Stocking Quantity'] = data['Bag Qty'] * round(data['Exact Stocking Quantity']/data['Bag Qty'])

In [22]:
refill_data = data[data['Stocking Quantity'] > 0]

In [23]:
refill_data.shape

(12, 18)

In [24]:
refill_sheet = refill_data[['Part Number', 'Inventory_x','Min', 'Max', 'Stocking Quantity']]
refill_sheet

,Part Number,Inventory_x,Min,Max,Stocking Quantity
0,48-321,15500,10000,20000,4500.0
1,48-323,33122,30000,45000,11900.0
2,48-324,14500,10000,15000,500.0
4,48-326,0,4000,6700,6700.0
5,48-328,8700,6000,9000,300.0
6,48-330,33800,30000,54700,20900.0
7,48-331,28730,30000,48000,19300.0
8,48-332,52500,50000,56700,4200.0
9,48-333,11000,10000,15000,4000.0
10,48-334,10500,8000,10700,200.0


In [25]:
refill_sheet.to_csv(save_path)